In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans

In [11]:
df = pd.read_pickle('data/latest_merged.pkl')

In [6]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)
pd.set_option("display.max_colwidth",None)

In [7]:
df.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,No_of_PreviousApplications,Credit_to_Income_Ratio,Annuity_to_Income_Ratio,GoodsPrice_to_Income_Ratio,No_of_DocumentsSubmitted,EXT_SCORES_AVG,EXT_SCORES_SUM,FLAG_CONTACT_INFO,EXT_SOURCE_1,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Security staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_XNA,clusters,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,HOUR_APPR_PROCESS_START_y_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,DAYS_DECISION_list,SELLERPLACE_AREA_list,CNT_PAYMENT_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list,NAME_CONTRACT_TYPE_y_list_Cash loans,NAME_CONTRACT_TYPE_y_list_Consumer loans,NAME_CONTRACT_TYPE_y_list_Revolving loans,NAME_CONTRACT_TYPE_y_list_XNA,WEEKDAY_APPR_PROCESS_START_y_list_FRIDAY,WEEKDAY_APPR_PROCESS_START_y_list_MONDAY,WEEKDAY_APPR_PROCESS_START_y_list_SATURDAY,WEEKDAY_APPR_PROCESS

In [8]:
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier

In [9]:
def split(dataframe):
    X = dataframe.drop('TARGET',axis=1)
    y = dataframe['TARGET']
    y = y.astype('int')

    X_train1,X_test,y_train1,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

    X_train,X_val,y_train,y_val = train_test_split(X_train1,y_train1,test_size=0.3,random_state=0)
    return X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val

def fit_predict(model):
    model.fit(X_train,y_train)

    y_pred = model.predict(X_val)

    print("Accuracy = {0} \nRecall {1} \nPrecision {2} \nF1 score {3}".format( 
        accuracy_score(y_val,y_pred),recall_score(y_val,y_pred),
        precision_score(y_val,y_pred),f1_score(y_val,y_pred)))

def cross_validate(model,scoring_metric,folds):  
    skf = StratifiedKFold(folds,shuffle=True,random_state=0)
    scores = cross_val_score(log,X_train1,y_train1,cv=skf,scoring=scoring_metric)

    print(scores)
    print(np.mean(scores))

def feature_importance(model):

    feature_imp =  pd.DataFrame(model.feature_importances_,index=list(df.columns.drop('TARGET')),
              columns=['feature_imp']).sort_values('feature_imp',ascending=False)
    imp_columns = list(feature_imp[feature_imp['feature_imp']>0].index)
    imp_df = df[imp_columns]
    imp_df['TARGET'] = df['TARGET']
    return feature_imp,imp_df

In [10]:
cat = df.select_dtypes(include='object').columns
lb = LabelEncoder()
for col in cat:
    df[col] = lb.fit_transform(df[col])

In [58]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(df)

In [10]:
log = LogisticRegression(random_state=42)
fit_predict(log)

Accuracy = 0.9189062229584395 
Recall 0.014889091461561836 
Precision 0.5632183908045977 
F1 score 0.02901124925991711


In [11]:
from sklearn.decomposition import PCA

In [79]:
pca = PCA(n_components=80,random_state=0)

In [76]:
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=80, random_state=0,
    svd_solver='auto', tol=0.0, whiten=False)

In [77]:
pca.explained_variance_ratio_,pca.explained_variance_

(array([0.08746314, 0.07449567, 0.05502724, 0.05219221, 0.04674045,
        0.03538295, 0.02952539, 0.02698752, 0.02664655, 0.02210167,
        0.02126086, 0.02006078, 0.01949904, 0.01804908, 0.0175435 ,
        0.01714417, 0.01553276, 0.01536355, 0.01503843, 0.01455189,
        0.01428576, 0.01412575, 0.01405591, 0.01379837, 0.01340935,
        0.01255943, 0.01148711, 0.01062473, 0.01020807, 0.00981684,
        0.00932366, 0.0090476 , 0.00897229, 0.00856766, 0.00782699,
        0.00767738, 0.00739705, 0.00718778, 0.00689367, 0.00598008,
        0.00587342, 0.00556706, 0.00500899, 0.00492948, 0.00460925,
        0.00452791, 0.00434083, 0.00431928, 0.00424471, 0.00406485,
        0.00404893, 0.00395765, 0.00386468, 0.00354851, 0.00330165,
        0.00326259, 0.00302988, 0.00282564, 0.00271873, 0.00263311,
        0.00262935, 0.00257775, 0.00257069, 0.00237477, 0.00232075,
        0.00231018, 0.00230708, 0.00228343, 0.00227041, 0.00221689,
        0.00219508, 0.00203957, 0.0019732 , 0.00

In [78]:
np.sum(pca.explained_variance_ratio_)

0.9572721797887973

In [80]:
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)

In [81]:
log = LogisticRegression(random_state=0)

log.fit(X_train_pca,y_train)
y_pred = log.predict(X_val_pca)

In [82]:
accuracy_score(y_val,y_pred)

0.9186589858333127

### SMOTE

In [66]:
from imblearn.over_sampling import SMOTE

In [67]:
smote = SMOTE(random_state=0)

In [68]:
X_train_smote,y_train_smote = smote.fit_sample(X_train,y_train)

In [69]:
log = LogisticRegression(random_state=0)
log.fit(X_train_smote,y_train_smote)
y_pred = log.predict(X_val)
accuracy_score(y_val,y_pred)

0.9187826043958761

### SelectPercentile

In [86]:
from sklearn.feature_selection import SelectPercentile,chi2,f_classif

In [109]:
selectp = SelectPercentile(f_classif,percentile=40)

In [110]:
X_train_chi = selectp.fit_transform(X_train,y_train)
X_val_chi = selectp.transform(X_val)

In [106]:
log = LogisticRegression(random_state=0)
log.fit(X_train_chi,y_train)
y_pred = log.predict(X_val_chi)
accuracy_score(y_val,y_pred)

0.9189309466709521

In [113]:
gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train_chi,y_train)
y_pred = gb.predict(X_val_chi)
accuracy_score(y_val,y_pred)

0.9192523549336168

### KS for train

In [59]:
gb = RandomForestClassifier(random_state=0)

# fit_predict(gb)

In [60]:
gb.fit(X_train,y_train)
y_pred = gb.predict(X_val)
accuracy_score(y_val,y_pred)

0.9171449998557789

In [61]:
pred_proba = gb.predict_proba(X_val)

In [66]:
X_val['proba'].describe()

count    69338.000000
mean         0.909887
std          0.073888
min          0.410000
25%          0.880000
50%          0.930000
75%          0.960000
max          1.000000
Name: proba, dtype: float64

In [67]:
X_val['bucket'].value_counts()

(62404.3, 69338.0]    6934
(55470.6, 62404.3]    6934
(48536.9, 55470.6]    6934
(34669.5, 41603.2]    6934
(27735.8, 34669.5]    6934
(13868.4, 20802.1]    6934
(6934.7, 13868.4]     6934
(0.999, 6934.7]       6934
(41603.2, 48536.9]    6933
(20802.1, 27735.8]    6933
Name: bucket, dtype: int64

In [65]:
X_val['proba'] = pd.DataFrame(pred_proba,index=list(y_val.index),columns=['proba',1])['proba']
X_val['TARGET'] = y_val
X_val['TARGET0'] = 1-X_val['TARGET']
X_val['bucket'] = pd.qcut(X_val['proba'].rank(method='first'),10)

In [68]:
grouped = X_val.groupby('bucket',as_index=False)

In [69]:
grouped.min()['proba']

0    0.41
1    0.81
2    0.86
3    0.89
4    0.91
5    0.93
6    0.94
7    0.96
8    0.97
9    0.98
Name: proba, dtype: float64

In [70]:
ks_df = pd.DataFrame()
ks_df['Model_score_starting'] = grouped.min().proba
ks_df['Model_score_ending'] = grouped.max().proba
ks_df['no of good customers'] = grouped.sum().TARGET0
ks_df['no of bad customers'] = grouped.sum().TARGET


In [71]:
ks_df

,Model_score_starting,Model_score_ending,no of good customers,no of bad customers
0,0.41,0.81,5159,1775
1,0.81,0.86,5985,949
2,0.86,0.89,6249,685
3,0.89,0.91,6383,550
4,0.91,0.93,6497,437
5,0.93,0.94,6530,404
6,0.94,0.96,6629,304
7,0.96,0.97,6666,268
8,0.97,0.98,6729,205
9,0.98,1.00,6764,170


In [72]:
ks_df.to_csv('KS/KS_train_df.csv')

In [73]:
len(y_train),len(y_val),len(y_test)

(161787, 69338, 57782)

In [46]:
len(df)

288907

### Ks for Test

In [74]:
list(X_val['bucket'].value_counts().index)

[Interval(62404.3, 69338.0, closed='right'),
 Interval(55470.6, 62404.3, closed='right'),
 Interval(48536.9, 55470.6, closed='right'),
 Interval(34669.5, 41603.2, closed='right'),
 Interval(27735.8, 34669.5, closed='right'),
 Interval(13868.4, 20802.1, closed='right'),
 Interval(6934.7, 13868.4, closed='right'),
 Interval(0.999, 6934.7, closed='right'),
 Interval(41603.2, 48536.9, closed='right'),
 Interval(20802.1, 27735.8, closed='right')]

In [77]:
pred_proba =  gb.predict_proba(X_test)

In [78]:
X_test['proba'] = pd.DataFrame(pred_proba,index=list(y_test.index),columns=['proba',1])['proba']
X_test['TARGET'] = y_test
X_test['TARGET0'] = 1-X_test['TARGET']
proba_bins = [0.41,0.81,0.86,0.89,0.91,0.93,0.94,0.96,0.97,0.98,1]
X_test['bucket'] = pd.cut(X_test['proba'],bins=proba_bins)

In [79]:
grouped = X_test.groupby('bucket',as_index=False)

In [80]:
grouped.min()['proba']

0    0.43
1    0.82
2    0.87
3    0.90
4    0.92
5    0.94
6    0.95
7    0.97
8    0.98
9    0.99
Name: proba, dtype: float64

In [81]:
ks_df = pd.DataFrame()
ks_df['Model_score_starting'] = grouped.min().proba
ks_df['Model_score_ending'] = grouped.max().proba
ks_df['no of good customers'] = grouped.sum().TARGET0
ks_df['no of bad customers'] = grouped.sum().TARGET


In [82]:
ks_df

,Model_score_starting,Model_score_ending,no of good customers,no of bad customers
0,0.43,0.81,4892,1521
1,0.82,0.86,5197,816
2,0.87,0.89,5516,572
3,0.90,0.91,5004,435
4,0.92,0.93,6522,421
5,0.94,0.94,3978,206
6,0.95,0.96,9004,383
7,0.97,0.97,4498,155
8,0.98,0.98,4081,128
9,0.99,1.00,4346,106


In [83]:
ks_df.to_csv('KS/KS_test_df.csv')

In [69]:
len(X_test)

57782